In [4]:
import pandas as pd
import numpy as np
from datetime import date
from scipy.sparse import coo_matrix
import requests

data_df = pd.read_csv('../../data/ratings.csv', sep=',',header=0, names=["UserID", "MovieID", "Rating", "Timestamp"],dtype={'Rating':'float64', "Timestamp":"float64"})
filtered_data = pd.read_csv('./../../data/filtered_ratings.csv', sep=',', names = ["UserID", "MovieID", "Rating", "Timestamp"])
meta_df = pd.read_csv('../../data/movies_metadata.csv',header=0, usecols=["id", "imdb_id", "original_title"])
data_df["Rating"] = (data_df["Rating"] > 0).astype(float)

In [5]:
## we want the k movies that have been rated the most times.
filtered_data.dropna()
k = 100

value_counts = filtered_data['MovieID'].value_counts()
movies_in_meta = meta_df["id"].unique()
print(value_counts)
movies = []
ids = []
for movie, _ in value_counts.iteritems():
    if(len(movies)>=k):
        break
    if(str(movie) in movies_in_meta):
        movies.append(meta_df[meta_df["id"] == str(movie)]["original_title"].item())
        ids.append(movie)


2571      1346
318       1333
79132     1189
2959      1150
356       1145
          ... 
158412       1
125628       1
641          1
160207       1
2049         1
Name: MovieID, Length: 19700, dtype: int64


In [6]:
api_results = []
IMDB_API_KEY_HERE = "k_i6ke8znc"
imdb_url = "https://imdb-api.com/en/API/SearchMovie/"+IMDB_API_KEY_HERE+"/"
for movie in movies:
    mov_url = imdb_url + movie
    res = requests.get(mov_url)
    api_results.append(res.json)

In [8]:
images = []
for result in api_results:
    try:
        res = result()
        # print(res['results'][0]['image'])
        images.append(res['results'][0]['image'])
    except:
        images.append("not found")

In [7]:
api_results[0]()

{'searchType': 'Movie',
 'expression': 'The Million Dollar Hotel',
 'results': [{'id': 'tt0120753',
   'resultType': 'Title',
   'image': 'https://imdb-api.com/images/original/MV5BNDQ2MGUwYjYtODFiNy00OTViLWJhOTgtZWNmM2I2MDRmMzQ1XkEyXkFqcGdeQXVyNjU0NTI0Nw@@._V1_Ratio0.7273_AL_.jpg',
   'title': 'The Million Dollar Hotel',
   'description': '(2000)'},
  {'id': 'tt0172464',
   'resultType': 'Title',
   'image': 'https://imdb-api.com/images/original/MV5BODY2MzUxMjczMF5BMl5BanBnXkFtZTYwOTc0NzI5._V1_Ratio0.7273_AL_.jpg',
   'title': 'The Million Dollar Kid',
   'description': '(2000)'},
  {'id': 'tt0037073',
   'resultType': 'Title',
   'image': 'https://imdb-api.com/images/original/MV5BOGVkN2QyZjEtMDMyMy00ZTUyLTlhYWEtOWMyYmJkZTI4ZjkxXkEyXkFqcGdeQXVyNjMxODMyODU@._V1_Ratio0.7273_AL_.jpg',
   'title': 'The Million Dollar Cat',
   'description': '(1944) (Short)'},
  {'id': 'tt0066728',
   'resultType': 'Title',
   'image': 'https://imdb-api.com/images/original/MV5BYWQyYTU0ZmQtNDE5ZS00M2E4LWIxZj

In [93]:
with open('top100images.json','w', encoding="utf-8") as file:
    file.write("{\n")
    for i in range(100):
        file.write('"')
        file.write(movies[i])
        file.write('"')
        file.write(':')
        file.write('"')
        file.write(images[i])
        file.write('"')
        if (i!=99):
            file.write(',')
        file.write('\n')
    file.write("}\n")

In [ ]:
with open('ignore.py', w, encoding="utf-8") as file:
    file.write('ignore = [')
    for i in range(100):
        file.write(str(ids[i]))
        if(i != 99):
            file.write(',')
    file.write(']')

In [10]:
with open('top100.js','w', encoding="utf-8") as file:
    file.write("export default const top100=[")
    for i in range(100):
        file.write("{")
        file.write("name: ")
        file.write('"')
        file.write(movies[i])
        file.write('"')
        file.write(',')

        file.write('image:')
        file.write('"')
        file.write(images[i])
        file.write('"')
        file.write(',')
        
        file.write('id: ')
        file.write('"')
        file.write(str(ids[i]))
        file.write('"')

        file.write('}')
        
        if (i!=99):
            file.write(',')
        file.write('\n')
    file.write("];\n")

In [99]:
meta_df[meta_df["original_title"] == "Солярис"]["id"].item()

ValueError: can only convert an array of size 1 to a Python scalar